# Email Activity (Sent/Receive) Report
- Replace variables with your tenant information
- Import required modules
- (Optional) Store your credentials
- Authenticate with Graph API
- Query the Graph API
- Clean up results and Output to Scatter Chart

## Set your variables here

In [16]:
############ This is the credentials used to authenticate with the Graph API ###################
$credentials = @{
    tenantDomain = "domain.com"
    tenantId = "xxxxxxx-xxxx-xxxx-xxxx-xxxxxxxx"        ## AAD Tenant ID
    client_id = "ae4fa870-23af-4ae5-a6ed-xxxxx"     ## Application Client ID
    client_secret = "x~xxxxxxx~xxxxx~xxxxxxxxx-xxx"   ## Application Client Secret
}

## Import required modules

In [20]:
$modules = @('pp-core','graph') ## These are the modules to import 
$credentialPath = "~/.credentials/graph.credential" ## Where to store credentials
################## Import Graph Authentication Module #############################
ForEach ($module in $modules) {
    Import-Module "~/Notebooks/Powershell-Playground/PowerShell/custom-modules/$module" -Force  ## import module 
    Get-Command -Module "$module"                                                        ## list commands in this module 
}

$credentials = New-StoreCredentials -credentialPath $credentialPath -credentialArray $credentials


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

 ·········


## Authenticate with Graph API and grab access token

In [25]:
## Get a token if authenticates properly.
$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = "https://graph.microsoft.com"    ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = "client_credentials"  ## This is using a appliation ID and secret to authenticate
};

$tokenResponse = Invoke-GraphAuthentication -authParams $authParams


Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/de40cf7e-ad5f-4245-a317-14be39cbb0ef/oauth2/token


Received Token!
Connected and Access Token received and will expire 


## Query the Graph API

In [22]:
## Request Parameters
$endpoint = "beta/reports/getEmailActivityCounts";
$filters = "(period='D$($howManyDays)')?`$format=application/json"
$URI = "$resource/$($endpoint)$($filters)"; Write-Host "URI:",$URI ## URL of where to query in the graph API/beta/reports/getEmailActivityCounts(period='D7')?$format=text/csv

$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" 
};

#query Graph API and insert into $graphresponse variable
$Result = (Invoke-RestMethod -Method Get -Uri $URI -Headers $authHeader -ErrorAction Stop).value
$Result | Select -First 5 | Format-Table

URI: https://graph.microsoft.com/beta/reports/getEmailActivityCounts(period='D7')?$format=application/json

@odata.type                           reportRefreshDate send receive read reportDate reportPeriod
-----------                           ----------------- ---- ------- ---- ---------- ------------
#microsoft.graph.emailActivitySummary 2020-06-25           2      21    1 2020-06-25 7
#microsoft.graph.emailActivitySummary 2020-06-25           4      23      2020-06-24 7
#microsoft.graph.emailActivitySummary 2020-06-25           2      20      2020-06-23 7
#microsoft.graph.emailActivitySummary 2020-06-25           2      49   11 2020-06-22 7
#microsoft.graph.emailActivitySummary 2020-06-25           2      53      2020-06-21 7



## Clean up any empty content in send and receive and generate E-mail Activity Report

In [23]:
ForEach ($item in $Result) {
    if ($item.send -eq $null) {
        $item.send = 0
    }
    if ($item.receive -eq $null) {
        $item.receive = 0
    }
}

$sendSeries = [Graph.Scatter]@{
    name = "Emails Sent"
    x = $Result.reportDate
    y = $Result.send
}

$receiveSeries = [Graph.Scatter]@{
    name = "Emails Received"
    x = $Result.reportDate
    y = $Result.receive
}

$chart = @($receiveSeries, $sendSeries) | New-PlotlyChart -Title "Emails Received vs Sent"
Out-Display $chart